## Import libraries

In [ ]:
import random

from tqdm.notebook import tqdm
import numpy as np
import pickle
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from embedding_model import EmbeddingModel

## Define initial variables

In [ ]:
IMGS_FOLDER = 'flickr_images/15K Nocturna Valencia Banco Mediolanum/'

EMBEDDINGS_PATH = 'flickr_images/15K_Nocturna_Valencia_Banco_Mediolanum_nobg_embeddings.pkl'

## Load model

In [ ]:
embed_model = EmbeddingModel()

## Load image embeddings

In [ ]:
# Load image embeddings from disk. Each item of the dict will contain 
# the name of the image as a key, and the numpy array of the embedding as a value

with open(EMBEDDINGS_PATH, 'rb') as handle:
    embed_dict = pickle.load(handle)

print('Number of items: ', len(embed_dict.keys()))
print(list(embed_dict.keys())[:5])

img_names_list = list(embed_dict.keys())
img_embeddings_np = np.array(list(embed_dict.values()))

print('embeddings_np.shape: ', img_embeddings_np.shape)

## Make a query

In [ ]:
text_search = 'a man with a blue t-shirt'

n_cols = 3
n_rows = 12
n_images = n_cols * n_rows

indexes = embed_model.get_similar_images_indexes(img_embeddings_np, text_search, n=n_images)

target_img_filenames = [img_names_list[index] for index in indexes]
target_img_paths = [(IMGS_FOLDER + img_name) for img_name in target_img_filenames]

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4*n_rows))
for ax, img in zip(fig.axes, target_img_paths):
    ax.imshow(Image.open(img))
    ax.axis('off')

plt.suptitle(text_search, fontsize=20)
plt.tight_layout()
plt.show()